# 单量子比特标定


*版权所有 (c) 2021 百度量子计算研究所，保留所有权利。*

## 内容概要

本教程介绍单量子比特频率、弛豫时间 $T_1$ 和失相时间 $T_2$ 的标定方法以及该量子比特上 $\pi$ 脉冲的校准。本教程的大纲如下：
- 背景介绍
- 准备工作
- 构建模拟器
- 量子比特频率标定
- Rabi 振荡校准 $\pi$ 脉冲
- 纵向弛豫标定 $T_1$
- Ramsey 振荡标定 $T_2$
- 总结

## 背景介绍

由于制造工艺的限制以及实际应用的需要，不同的超导量子比特具有不同的频率、相干时间等特性。因此我们需要对这些参数进行标定，即对量子比特执行一系列操作，并进行测量，从测量结果中获取关于此量子比特的信息，如量子比特频率以及相干时间 $T_1$、$T_2$ 等。其中，量子比特的频率为实现单量子比特门的脉冲信号的驱动频率；相干时间为量子比特保持其信息的持续时间，相干时间越长，量子比特的质量越好，可进行运算的时间就越长。

## 准备工作

在运行此教程前，您首先需要从量脉（Quanlse）和其他常用 Python 库导入必要的包。

In [ ]:
from Quanlse.Simulator.PulseSim1Q import pulseSim1Q
from Quanlse.Calibration.SingleQubit import qubitSpec, ampRabi, fitRabi, longRelax, ramsey, fitRamsey

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (7, 5)

from numpy import array, pi, exp
from scipy.signal import find_peaks

## 构建模拟器

在进行标定演示前，我们首先需要构建一个单量子比特模拟器作为标定对象。在 Quanlse v2.1 中我们内置了设定好参数的单量子比特模拟器 `pulseSim1Q()` （相关参数如量子比特频率以及 $T_1$ 和 $T_2$ 等可进行自定义）。`pulseSim1Q()` 函数需要两个参数：`dt` 表示求解模拟演化时的步长，而 `frameMode` 则表示采用何种坐标系进行仿真（`'lab'`、`'rot'` 分别表示实验室坐标系和旋转坐标系）。完成初始化后，我们将该模拟器视为 "黑箱" 进行标定的演示。

In [ ]:
# AWG sampling time
dt = 0.01

# Instantiate the simulator object
model = pulseSim1Q(dt=dt, frameMode='lab')

# Define system parameters
model.qubitFreq = {0: 5.212 * (2 * pi)}
model.T1 = {0: 2000}
model.T2 = {0: 600}

## 量子比特频率标定

在标定量子比特的其它参数之前，我们首先需要确定量子比特的频率。量子比特频率确定后，我们就可以正确地设置本机振荡器（Local Oscillator）频率，从而使得施加的脉冲与量子比特共振。

为了测量量子比特频率，我们利用外加脉冲与量子比特共振激发的原理，改变本机振荡器频率，对量子比特施加一定振幅的脉冲。量子比特被最大程度的激发时的脉冲频率，即为量子比特的频率。而在实际的实验中，量子比特频率的大致范围会提供给实验人员。因此，我们可以在给定的范围内进行频率扫描，并确定较为精确的量子比特频率。

我们首先在较大的频率范围内（4.6 GHz 到 5.8 GHz）进行扫描。具体的方法为使用校准模块 `Quanlse.Calibration.SingleQubit` 中的函数 `qubitSpec()`，并输入脉冲模型 `pulseModel`、频率范围 `frequeRange`、样本数量 `sample`、脉冲幅度 `amp` 和脉冲持续时间 `t` 。在完成扫描后，该函数将返回扫描频率和对应的激发态布居数：

In [ ]:
# Define frequency range
freqRange = [4.1 * (2 * pi), 5.9 * (2 * pi)]

# Scan qubit frequency spectrum
freqList, popList = qubitSpec(pulseModel=model, freqRange=freqRange, sample=50, amp=0.9, t=20)

激发态布居数与本机振荡器频率关系图如下。

In [ ]:
# Convert unit
freq = [x / (2 * pi) for x in freqList]

# Plot population graph
plt.plot(freq, popList)
plt.title("Frequency spectrum", size=17)
plt.xlabel("LO frequency (GHz)", size=15)
plt.ylabel(r"$|1\rangle$ population)", size=15)
plt.show()

从图中我们可以看到量子比特频率大致在 5.1 GHz 和 5.3 GHz 之间。接下来我们缩小扫描范围进行第二次扫描，并绘制激发态布居数与本机振荡器频率关系图。

In [ ]:
# Define new frequency range
nFreqRange = [5.1 * (2 * pi), 5.3 * (2 * pi)]

# Scan qubit frequency spectrum
nFreqList, nPopList = qubitSpec(model, nFreqRange, 30, 0.9, 20)

In [ ]:
# Convert unit
nFreq = [x / (2 * pi) for x in nFreqList]

# Plot population graph
plt.plot(nFreq, nPopList)
plt.title("Frequency spectrum", size=17)
plt.xlabel("LO frequency (GHz)", size=15)
plt.ylabel(r"$|1\rangle$ population)", size=15)
plt.show()

然后，我们使用 `scipy` 中的函数 `find_peak()` 来寻找峰值所对应的频率。

In [ ]:
# Find peak
peak = find_peaks(nPopList, height=0.3)[0][0]
qubitFreq = nFreq[peak]

# Plot peak
plt.plot(nFreq, nPopList)
plt.title(f'Qubit frequency: {round(qubitFreq, 6)} GHz', size=17)
plt.plot(nFreq[peak], nPopList[peak], 'x', mfc=None, mec='red', mew=2, ms=8)
plt.xlabel('Frequency (GHz)', size=15)
plt.ylabel(r'$|1\rangle$ population', size=15)
plt.show()

如上图所示，我们标定得到的量子比特频率为 5.217241 GHz。

## Rabi 振荡校准 $\pi$ 脉冲

在确定了量子比特的频率后，我们可以校准 $\pi$ 和 $\pi/2$ 脉冲的波形参数。为此，我们进行 Rabi 振荡实验。通常有两种方式进行 Rabi 振荡：确定其他参数不变，固定脉冲振幅扫描脉冲持续时间或固定脉冲持续时间扫描脉冲振幅。选择适当的范围后，激发态（或基态）的布居数将以正弦波的形式振荡。为进行上述实验，我们从 `Quanlse.Calibration.SingleQubit` 模块导入函数 `ampRabi()`，并输入参数：脉冲模型 `pulseModel`、振幅范围 `ampRange`、脉冲持续时间 `tg` 和样本数量 `sample` 。该函数将返回扫描振幅和相应的激发态布居数列表。

另外，`calibration` 模块还包括了通过扫描脉冲的时间的函数 `tRabi()`。该函数通过固定脉冲幅值并且改变脉冲的时间来实现 Rabi 振荡，因此用法与 `ampRabi()` 非常类似。

In [ ]:
# Define amplitude range
ampRange = [0, 6]

# Scan different amplitudes for Rabi oscillation
ampList, popList = ampRabi(pulseModel=model, pulseFreq=qubitFreq * 2 * pi, 
                           ampRange=ampRange, tg=20, sample=50)

激发布居数与脉冲振幅关系图如下：

In [ ]:
# Plot Rabi Oscillation with different amplitudes
plt.plot(ampList, popList, '.')
plt.title("Rabi Oscillation", size=17)
plt.xlabel('Amplitude', size=15)
plt.ylabel(r'$|1\rangle$ population', size=15)
plt.show()

在得到布居数的分布之后，我们从 `Quanlse.Calibration.SingleQubit` 模块导入函数 `fitRabi()` 进行图像拟合，并获得能够实现 $\pi$ 和 $\pi/2$ 旋转的脉冲振幅。我们输入 `ampList` 作为 X 轴，并同时输入布居数 `popList` 作为 Y 轴进行拟合，其中拟合函数的形式为：$y=a\cdot \cos（b\cdot x+c）+d$。最终，`fitRabi()` 将返回 $\pi/2$ 和 $\pi$ 脉冲的振幅：

In [ ]:
# Fit Rabi
halfPiAmp, piAmp = fitRabi(popList=popList, xList=ampList)
print("Pi/2-pulse amplitude: ", halfPiAmp)
print("Pi-pulse amplitude: ", piAmp)

## 纵向弛豫标定 $T_1$

得到 $\pi$ 和 $\pi/2$ 脉冲的参数后，我们可以进一步标定量子比特的相干时间 $T_1$ 和 $T_2$。我们首先进行 $T_1$ 的标定，将 $\pi$ 脉冲施加到量子比特上，并找到激发态布居数衰减到 $1/e$ 的时间 \[1\]。

为了将量子比特激发到激发态并观察其纵向弛豫，我们可以使用 `Quanlse.Calibration.SingleQubit` 模块中的 `longRelax()` 函数。输入参数：模拟器对象 `pulseModel`、AWG 采样时间 `dt`、脉冲频率 `pulseModel`、$\pi$ 脉冲幅度 `piAmp` 和持续时间 `piLen`、最大闲置时间 `maxIdle` 和拟合函数的初始值 `initFit`。随后，运行该函数进行模拟仿真，同时该函数将使用拟合函数 $y=e^{-x/T_1}$ 进行曲线拟合。最终返回 $T_1$、闲置时间、布居数仿真结果以及拟合结果的列表：

In [ ]:
# Longitudinal relaxation on a qubit
T1, tList, experimental, fitted = longRelax(pulseModel=model, dt=dt, pulseFreq=qubitFreq * 2 * pi, 
                                            piAmp=piAmp, piLen=20, maxIdle=4000, initFit=[1500])

$T_1$ 以及布居数随闲置时间变化的图像如下：

In [ ]:
# Print estimated T1
print("Estimated T1: ", T1, "ns")

# Plot fit result
plt.plot(tList, experimental, "+", label="Experiment")
plt.plot(tList, fitted, "r", label="Fitted", linewidth=2.)
plt.legend()
plt.xlabel("Idling time", size=15)
plt.ylabel(r'$|1\rangle$ population', size=15)
plt.title("Longitudinal Relaxation", size=17)
plt.show()

## Ramsey 振荡标定 $T_2$

在本节中，我们将使用 Ramsey 振荡实验进行失相时间 $T_2$ 的标定。首先，我们在量子比特上输入一个与量子比特频率相差非常小的驱动频率的 $\pi/2$ 脉冲，在等待闲置时间 $t_{\rm idle}$ 之后，再输入另一个 $\pi/2$ 脉冲，并测量量子比特的激发态布居数 \[2\]。此时，测量结果取决于闲置时间 $t_{\rm idle}$ 之后量子态的相位。

为进行 Ramsey 实验，我们从 `Quanlse.Calibration.SingleQubit` 模块导入函数 `Ramsey()`，输入参数：模拟器对象 `pulseModel`、脉冲频率 `pulseFreq` 、$\pi/2$ 脉冲持续时间 `tg` 、$\pi/2$ 脉冲幅度 `x90` 、采样数 `sample` 、最大闲置时间 `maxTime` 和脉冲频率与比特频率的失调 `detuning`（该程序运行时间可能会比较久，可以选择减少采样点以及减少运行时间，但是模拟的效果可能也随之下降）：

In [ ]:
# Scan different idle time for Ramsey oscillation
tList, popList = ramsey(pulseModel=model, pulseFreq=5.21 * 2 * pi, tg=20, x90=1.013,
                        sample=50, maxTime=600, detuning=0.07)

该函数返回闲置时间和相应的布居数列表。我们可以使用函数 `fitRamsey()` 来对数据进行拟合，输入参数 $T_1$ `t1`、布居数列表 `popList`、闲置时间列表 `tList` 以及失调 `detuning`，然后，使用函数 $y=\frac{1}{2} \cos（a\cdot x）e^{-b\cdot x}+0.5$ 拟合曲线。根据拟合结果，我们使用表达式 $T_2 = \frac{1}{(b-\frac{1}{2a})}$ 求得 $T_2$：

In [ ]:
# Fit Ramsey
T2, fitted = fitRamsey(t1=2000, popList=popList, tList=tList, detuning=0.07)

`fitRamsey()` 返回测得的 $T_2$ 值和拟合的布居数列表，$T_2$ 以及激发态布居数与闲置时间关系图如下：

In [ ]:
# Print estimated T2
print("Estimated T2: ", T2, " ns")

# Plot fit result
plt.plot(tList, popList, '.')
plt.plot(tList, fitted)
plt.plot(tList, list(exp(- (1 / 600 + 1 / (2 * 2000)) * array(tList)) * 0.5 + 0.5))
plt.xlabel("Idling time (ns)", size=15)
plt.ylabel(r"$|1\rangle$ population", size=15)
plt.title("Ramsey Experiment", size=17)
plt.show()

## 总结

本教程了介绍如何使用量脉进行单量子比特的频率、$\pi$ 脉冲、弛豫时间 $T_1$以及失相时间 $T_2$ 的标定。在阅读此教程后，用户可以通过这个链接 [tutorial-single-qubit-calibration-cn.ipynb](https://github.com/baidu/Quanlse/blob/main/Tutorial/CN/tutorial-single-qubit-calibration.ipynb) 跳转到此 Jupyter Notebook 文档对应的 GitHub 页面获取相关的代码。我们鼓励用户尝试不同于本教程的参数值以获得最佳结果。

## 参考文献

\[1\] [Krantz, Philip, et al. "A quantum engineer's guide to superconducting qubits." *Applied Physics Reviews* 6.2 (2019): 021318.](https://doi.org/10.1063/1.5089550)

\[2\] [Ramsey, Norman F. "A molecular beam resonance method with separated oscillating fields." *Physical Review* 78.6 (1950): 695.](https://doi.org/10.1103/PhysRev.78.695)